In [48]:
pip install openpyxl

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from urllib.parse import urljoin
import time
import math
from collections import defaultdict
from nltk import download
import openpyxl

# Download necessary NLTK data files (if not already installed)
download('wordnet')
download('stopwords')

class MultiPageSearchEngine:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.page_indexes = {}
        self.global_index = defaultdict(set)
        self.word_frequency = defaultdict(int)  # Tracks overall word frequency
        self.total_documents = 0

    def fetch_page_in_chunks(self, url):
      """Fetch page in chunks to handle truncation."""
      try:
          headers = {
              "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
              "Range": "bytes=0-"
          }
          response = requests.get(url, headers=headers, timeout=10, stream=True)

          if response.status_code in [200, 206]:
              content = b""
              for chunk in response.iter_content(chunk_size=1024):
                  content += chunk

              return BeautifulSoup(content.decode('utf-8', errors='ignore'), 'html.parser')
          else:
              print(f"Failed to fetch {url}: Status code {response.status_code}")
              return None
      except Exception as e:
          print(f"Error fetching {url}: {str(e)}")
          return None

    def get_links(self, soup, base_url):
        """Extract links from the page."""
        links = set()
        if soup:
            for link in soup.find_all('a', href=True):
                url = link['href']
                absolute_url = urljoin(base_url, url)
                # Allow more comprehensive link filtering
                if absolute_url.startswith(base_url) and re.search(r'/app/|/game/', absolute_url):
                    links.add(absolute_url)
        return links

    def clean_text(self, text):
        """Remove stopwords, non-alphanumeric characters, and apply lemmatization."""
        words = re.findall(r'\w+', text.lower())
        filtered_words = [self.lemmatizer.lemmatize(word) for word in words if word not in self.stop_words]
        return filtered_words

    def index_words(self, soup, url):
        """Create word index for a page and calculate term frequencies."""
        index = defaultdict(int)
        if soup:
            main_content = soup.get_text()
            words = self.clean_text(main_content)
            for word in words:
                index[word] += 1
                self.global_index[word].add(url)
                self.word_frequency[word] += 1  # Update global word frequency
        return dict(index)

    def crawl_and_index_query_based(self, start_url, query, max_relevant_pages=20):
        """Crawl pages based on query relevance."""
        visited = set()
        to_visit = {start_url}
        relevant_pages = 0
        query_terms = set(self.clean_text(query))

        while to_visit and relevant_pages < max_relevant_pages:
            url = to_visit.pop()
            if url in visited:
                continue

            print(f"Crawling: {url}")
            soup = self.fetch_page_in_chunks(url)
            if soup:
                # Index the current page
                page_index = self.index_words(soup, url)
                visited.add(url)

                # Check if the page contains the query terms
                if query_terms.intersection(page_index.keys()):
                    self.page_indexes[url] = page_index
                    relevant_pages += 1
                    self.total_documents += 1
                    print(f"Relevant page found: {url}")

                # Add new links to visit
                links = self.get_links(soup, start_url)
                to_visit.update(links - visited)

        print(f"\nCrawling complete. Indexed {len(self.page_indexes)} relevant pages.")
        return visited

    def compute_tf(self, term, page_index):
        """Calculate normalized term frequency (TF)."""
        term_count = page_index.get(term, 0)
        total_terms = sum(page_index.values())
        if total_terms == 0:
            return 0
        return term_count / total_terms

    def compute_idf(self, term):
        """Calculate smoothed inverse document frequency (IDF)."""
        doc_count = len(self.global_index[term])
        if doc_count > 0:
            return math.log((self.total_documents / doc_count) + 1)
        return 0

    def search(self, query, max_relevant_pages=20):
      """Search across all indexed pages using enhanced TF-IDF for all query concepts."""
      query_terms = set(self.clean_text(query))  # Clean and extract unique query terms
      results = defaultdict(float)

      for term in query_terms:
          if term in self.global_index:
              idf = self.compute_idf(term)  # IDF for the term
              for url in self.global_index[term]:
                  page_index = self.page_indexes.get(url, {})
                  tf = self.compute_tf(term, page_index)  # TF for the term on this page
                  results[url] += tf * idf  # Add TF-IDF score for this term to the page's score

      # Sort by aggregated TF-IDF score in descending order
      sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
      return sorted_results[:max_relevant_pages]


    def print_results(self, results):
        """Print search results in a formatted way."""
        if not results:
            print("No results found.")
            return

        print("\nSearch Results:")
        print("-" * 50)
        for url, score in results:
            print(f"URL: {url}")
            print(f"Score: {score:.4f}")
            print("-" * 50)

    def get_most_common_words(self, top_n=15):
        """Get the most common words across all indexed pages."""
        sorted_words = sorted(self.word_frequency.items(), key=lambda x: x[1], reverse=True)
        return sorted_words[:top_n]

    def build_inverted_index(self, most_common_words, max_pages_per_word=20):
        """Build an inverted index for the most common words."""
        inverted_index = defaultdict(list)

        for word, _ in most_common_words:
            pages = list(self.global_index[word])
            inverted_index[word] = pages[:max_pages_per_word]  # Limit to the first 20 pages

        return inverted_index

    def save_to_excel(self, visited_pages, most_common_words, search_results, filename="search_results_HW2.xlsx"):
        """Save visited pages, most common words, and search results to an Excel file."""
        wb = openpyxl.Workbook()

        # Sheet 1: Visited Pages
        ws1 = wb.active
        ws1.title = "Visited Pages"
        ws1.append(["Visited Pages"])
        for page in visited_pages:
            ws1.append([page])

        # Sheet 2: Most Common Words
        ws2 = wb.create_sheet(title="Most Common Words")
        ws2.append(["Word", "Frequency"])
        for word, frequency in most_common_words:
            ws2.append([word, frequency])

        # Sheet 3: Search Results
        ws3 = wb.create_sheet(title="Search Results")
        ws3.append(["URL", "Score"])
        for url, score in search_results:
            ws3.append([url, score])

        # Save the Excel file
        wb.save(filename)
        print(f"\nResults saved to {filename}")

def main():
    search_engine = MultiPageSearchEngine()
    start_url = 'https://store.steampowered.com/'
    # Excel_name = "/content/drive/My Drive/Colab Notebooks/Data Retrieval/HW2/search_results_HW2.xlsx"
    query = input("Enter your search query: ")
    print("Crawling and indexing pages based on query '" + query + "' ...")

    visited_pages = search_engine.crawl_and_index_query_based(
        start_url,
        query,
        max_relevant_pages = 20
    )

    print(f"\nIndexed {len(visited_pages)} pages.")
    for page in visited_pages:
        print(f"- {page}")

    # Get 15 most common words and build the inverted index
    most_common_words = search_engine.get_most_common_words()
    inverted_index = search_engine.build_inverted_index(most_common_words)

    print("\nTop 15 Most Common Words and Indexed Pages:")
    for word, pages in inverted_index.items():
        print(f"\nWord: {word}")
        print(f"Pages: {len(pages)}")
        for page in pages:
            print(f"- {page}")

    # Perform search
    results = search_engine.search(query, max_relevant_pages=20)
    search_engine.print_results(results)

    # Save outputs to an Excel file
    search_engine.save_to_excel(visited_pages, most_common_words, results)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Enter your search query: Very Positive
Crawling and indexing pages based on query 'Very Positive' ...
Crawling: https://store.steampowered.com/
Crawling: https://store.steampowered.com/app/2522740/HoneySun/?snr=1_4_661__105
Crawling: https://store.steampowered.com/app/3132990/Black_Myth_Wukong_Benchmark_Tool/?snr=1_4_661__tab-TrendingFree
Relevant page found: https://store.steampowered.com/app/3132990/Black_Myth_Wukong_Benchmark_Tool/?snr=1_4_661__tab-TrendingFree
Crawling: https://store.steampowered.com/app/3062170/The_Enigma_Cases/?snr=1_4_661__105
Crawling: https://store.steampowered.com/app/2699820/Shin_chan_Shiro_and_the_Coal_Town/?snr=1_4_661__145
Relevant page found: https://store.steampowered.com/app/2699820/Shin_chan_Shiro_and_the_Coal_Town/?snr=1_4_661__145
Crawling: https://store.steampowered.com/app/1364780/Street_Fighter_6/?snr=1_4_661__104
Relevant page found: https://store.steampowered.com/app/1364780/Street_Fighter_6/?snr=1_4_661__104
Crawling: https://store.steampowere